In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
b_value = np.array([0.0,0.25,0.5,1.0]).reshape(-1,1)


LR_tune, B_value = np.meshgrid(lr_tune,b_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
B_value = B_value.flatten('F').reshape(-1,1)


lrb_tune = np.hstack((LR_tune,B_value))

In [6]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [7]:
u0 = 50.0
loss_thresh = 0.1

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [8]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,beta_init):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        
        self.beta = Parameter(beta_init*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            z1 = self.activation(z) 
            a = z1 + self.beta[:,i]*z*z1
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
nan_tune = []
for tune_reps in range(20):
    label = "inv_HT_stan_tune" + str(tune_reps)
    max_reps = 10 #10
    max_iter = 75 #75

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    beta_full = []
   
    lambda1_full = []
    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    
    beta_init = lrb_tune[tune_reps,1]

    for reps in range(max_reps):
        print(label)
        'Generate Training data'
        print(reps)
        torch.manual_seed(reps*36)
        
        train_loss = []
        test_mse_loss = []
        test_re_loss = []   
        beta_val = []
        
        lambda1_val = []

        N_f = 50000 #Total number of collocation points 
        N_train = 500

        layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        
        PINN = Sequentialmodel(layers,beta_init)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrb_tune[tune_reps,0], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')
        
        
        nan_flag = train_model(max_iter,reps)

        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        beta_full.append(beta_val)

        lambda1_full.append(lambda1_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full,"lambda1": lambda1_full, "label": label}
    savemat(label+'.mat', mdic) 


inv_HT_stan_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 838.29047 Test MSE 858.052231756559 Test RE 0.4931125244857968 Lambda1 -0.04077347
1 Train Loss 838.0643 Test MSE 858.2624949987687 Test RE 0.49317293869094725 Lambda1 -0.040838197
2 Train Loss 838.0606 Tes

0 Train Loss 838.5584 Test MSE 858.0673331218202 Test RE 0.49311686375519515 Lambda1 -0.045517873
1 Train Loss 838.03406 Test MSE 858.1834192835424 Test RE 0.4931502190093856 Lambda1 -0.04571812
2 Train Loss 838.0248 Test MSE 858.2525266353397 Test RE 0.49317007468291385 Lambda1 -0.045733
3 Train Loss 837.99133 Test MSE 858.3246163173344 Test RE 0.4931907863768592 Lambda1 -0.045677032
4 Train Loss 837.86426 Test MSE 857.9083533103584 Test RE 0.49307118012725726 Lambda1 -0.051355284
5 Train Loss 837.8577 Test MSE 857.852766159421 Test RE 0.49305520588828683 Lambda1 -0.054048974
6 Train Loss 837.8537 Test MSE 857.8300561966475 Test RE 0.49304867951224657 Lambda1 -0.059168454
7 Train Loss 837.8245 Test MSE 857.7935742123682 Test RE 0.493038195159727 Lambda1 -0.09518738
8 Train Loss 837.2868 Test MSE 857.3657799538722 Test RE 0.492915237131654 Lambda1 -0.48834583
9 Train Loss 836.5898 Test MSE 855.1002057334423 Test RE 0.4922635463390581 Lambda1 -0.55545044
10 Train Loss 830.1159 Test MSE 

1 Train Loss 838.0571 Test MSE 858.2035491140375 Test RE 0.4931560027217493 Lambda1 -0.13790658
2 Train Loss 838.0347 Test MSE 858.3112676588329 Test RE 0.49318695131209767 Lambda1 -0.13802315
3 Train Loss 837.8281 Test MSE 857.7986319560845 Test RE 0.49303964868959277 Lambda1 -0.1390356
4 Train Loss 837.8052 Test MSE 857.7224859838985 Test RE 0.49301776487258864 Lambda1 -0.13934816
5 Train Loss 837.74603 Test MSE 857.6601007813348 Test RE 0.49299983508007483 Lambda1 -0.13998228
6 Train Loss 836.5897 Test MSE 855.2095156933369 Test RE 0.49229500907793156 Lambda1 -0.13045202
7 Train Loss 831.87164 Test MSE 846.6603369156414 Test RE 0.4898281930068563 Lambda1 0.044979576
8 Train Loss 764.7346 Test MSE 765.2232890789355 Test RE 0.46567536125313075 Lambda1 0.012346218
9 Train Loss 729.2855 Test MSE 718.809888280412 Test RE 0.4513320679843924 Lambda1 -0.00248334
10 Train Loss 716.55865 Test MSE 710.4272953240401 Test RE 0.4486926858959548 Lambda1 -0.0018971446
11 Train Loss 696.7299 Test MS

1 Train Loss 838.0614 Test MSE 858.3573544349697 Test RE 0.49320019189996556 Lambda1 -0.043526206
2 Train Loss 838.0543 Test MSE 858.2924653335411 Test RE 0.49318154936025194 Lambda1 -0.043521643
3 Train Loss 838.04974 Test MSE 858.2376898932253 Test RE 0.49316581191140285 Lambda1 -0.043521218
4 Train Loss 838.02734 Test MSE 858.0882708308769 Test RE 0.49312287999310733 Lambda1 -0.04356239
5 Train Loss 837.9049 Test MSE 857.8576342344968 Test RE 0.49305660486165875 Lambda1 -0.04595989
6 Train Loss 837.8946 Test MSE 857.9106680588648 Test RE 0.4930718453119494 Lambda1 -0.047422234
7 Train Loss 837.8932 Test MSE 857.9298560066286 Test RE 0.49307735927993207 Lambda1 -0.048802134
8 Train Loss 837.8726 Test MSE 858.0319055181053 Test RE 0.49310668382613415 Lambda1 -0.076861724
9 Train Loss 837.7972 Test MSE 857.7433237857185 Test RE 0.4930237536068532 Lambda1 -0.14603963
10 Train Loss 837.71173 Test MSE 857.2888705895488 Test RE 0.4928931283371084 Lambda1 -0.27991188
11 Train Loss 836.1853 

2 Train Loss 838.0585 Test MSE 858.2532796284802 Test RE 0.49317029102576765 Lambda1 -0.022164248
3 Train Loss 838.0578 Test MSE 858.2641169988377 Test RE 0.4931734047058129 Lambda1 -0.022164343
4 Train Loss 838.0575 Test MSE 858.2693657200412 Test RE 0.49317491270649566 Lambda1 -0.02216409
5 Train Loss 838.057 Test MSE 858.2791219916163 Test RE 0.49317771575114633 Lambda1 -0.022162851
6 Train Loss 838.05597 Test MSE 858.2987984312683 Test RE 0.49318336888058667 Lambda1 -0.022157675
7 Train Loss 838.0556 Test MSE 858.3025281022092 Test RE 0.4931844404244854 Lambda1 -0.022155944
8 Train Loss 838.0519 Test MSE 858.3329933047831 Test RE 0.49319319306709525 Lambda1 -0.022131393
9 Train Loss 837.8702 Test MSE 857.8979844886422 Test RE 0.49306820044849065 Lambda1 -0.020817071
10 Train Loss 837.8638 Test MSE 857.8136799582966 Test RE 0.4930439732646795 Lambda1 -0.02141558
11 Train Loss 837.7481 Test MSE 857.6300431070327 Test RE 0.4929911961342414 Lambda1 -0.07537261
12 Train Loss 837.68066 T

...

5 Train Loss 644.27966 Test MSE 646.3401782190261 Test RE 0.42797632726086776 Lambda1 0.071067065
6 Train Loss 561.14954 Test MSE 560.2435736633107 Test RE 0.39845346920430436 Lambda1 0.057910666
7 Train Loss 469.30032 Test MSE 469.95026267910293 Test RE 0.36493466515928197 Lambda1 0.054904133
8 Train Loss 386.30353 Test MSE 385.0510301877527 Test RE 0.3303302267443889 Lambda1 0.045112655
9 Train Loss 316.90707 Test MSE 317.1933469723819 Test RE 0.29981351453064875 Lambda1 0.024322832
10 Train Loss 284.6562 Test MSE 295.82186126295505 Test RE 0.28953715537382013 Lambda1 0.017776133
11 Train Loss 270.18848 Test MSE 282.4062151166179 Test RE 0.2828956671284295 Lambda1 0.015861861
12 Train Loss 264.2025 Test MSE 280.10621605077705 Test RE 0.2817413193333024 Lambda1 0.010592601
13 Train Loss 259.03616 Test MSE 278.25727525906257 Test RE 0.28080991283194173 Lambda1 0.0065539535
14 Train Loss 255.9763 Test MSE 275.6889146210112 Test RE 0.2795109475146735 Lambda1 0.008826247
15 Train Loss 252

6 Train Loss 323.1372 Test MSE 323.7151090052221 Test RE 0.30288004116855677 Lambda1 0.046953663
7 Train Loss 273.829 Test MSE 277.197546499675 Test RE 0.28027467756646623 Lambda1 0.0353465
8 Train Loss 255.20036 Test MSE 267.702222826664 Test RE 0.27543248428574424 Lambda1 0.038151246
9 Train Loss 244.26453 Test MSE 261.5688114725824 Test RE 0.27225894051357263 Lambda1 0.044589423
10 Train Loss 235.33165 Test MSE 251.06521115741535 Test RE 0.2667364949772064 Lambda1 0.05919022
11 Train Loss 227.53583 Test MSE 240.79950948015613 Test RE 0.2612263423767444 Lambda1 0.07985425
12 Train Loss 216.6905 Test MSE 227.94462418807606 Test RE 0.2541580371477362 Lambda1 0.10803818
13 Train Loss 205.34724 Test MSE 211.9911479159761 Test RE 0.2451026653047546 Lambda1 0.1439614
14 Train Loss 191.74239 Test MSE 200.27474910326413 Test RE 0.2382331914891755 Lambda1 0.17645156
15 Train Loss 178.6019 Test MSE 181.52456149442196 Test RE 0.2268072165342803 Lambda1 0.22312887
16 Train Loss 170.95306 Test MS

8 Train Loss 255.86833 Test MSE 273.9608978460049 Test RE 0.2786335838965292 Lambda1 0.030513356
9 Train Loss 246.55905 Test MSE 264.7176404062359 Test RE 0.2738927976840919 Lambda1 0.047177687
10 Train Loss 238.81384 Test MSE 256.236510677867 Test RE 0.2694695371841065 Lambda1 0.06653066
11 Train Loss 229.22188 Test MSE 246.17301930315423 Test RE 0.2641249311050275 Lambda1 0.09440562
12 Train Loss 213.85452 Test MSE 229.49581744851298 Test RE 0.25502136037903805 Lambda1 0.1473611
13 Train Loss 203.82324 Test MSE 217.08355146685815 Test RE 0.24802909556405428 Lambda1 0.18483512
14 Train Loss 192.94518 Test MSE 204.90502161956869 Test RE 0.24097138374427698 Lambda1 0.23044221
15 Train Loss 184.37244 Test MSE 197.293508888201 Test RE 0.23645340319949093 Lambda1 0.25624272
16 Train Loss 174.07663 Test MSE 183.52717946390695 Test RE 0.22805487783143338 Lambda1 0.29358146
17 Train Loss 165.2236 Test MSE 170.9870204283827 Test RE 0.22012569637063903 Lambda1 0.32436666
18 Train Loss 157.53226

9 Train Loss 259.3395 Test MSE 283.50926079744244 Test RE 0.2834476074117101 Lambda1 0.00057867594
10 Train Loss 256.36237 Test MSE 280.8151777231839 Test RE 0.28209764410409055 Lambda1 0.00159918
11 Train Loss 255.12268 Test MSE 279.6553962087572 Test RE 0.281514502295354 Lambda1 0.0030023619
12 Train Loss 253.62755 Test MSE 277.89054443608455 Test RE 0.28062480427721537 Lambda1 0.0059987004
13 Train Loss 249.7857 Test MSE 270.800465506682 Test RE 0.27702175347621566 Lambda1 0.020461852
14 Train Loss 243.8411 Test MSE 263.81182695110954 Test RE 0.27342379157806235 Lambda1 0.034637034
15 Train Loss 239.52567 Test MSE 257.81468616908643 Test RE 0.27029810258042053 Lambda1 0.049233116
16 Train Loss 233.30986 Test MSE 249.07132402602923 Test RE 0.2656752117135589 Lambda1 0.07289568
17 Train Loss 226.96779 Test MSE 241.91290184069797 Test RE 0.2618295661737941 Lambda1 0.09293388
18 Train Loss 221.49301 Test MSE 235.51710209817105 Test RE 0.25834519895908786 Lambda1 0.10849855
19 Train Loss

10 Train Loss 241.68968 Test MSE 260.86196564483316 Test RE 0.2718908245984959 Lambda1 0.09196885
11 Train Loss 234.03899 Test MSE 254.68123916270434 Test RE 0.26865049664437723 Lambda1 0.11293759
12 Train Loss 227.9635 Test MSE 248.57272135520935 Test RE 0.2654091579407334 Lambda1 0.13416247
13 Train Loss 221.65056 Test MSE 242.14362024668623 Test RE 0.2619543931325507 Lambda1 0.153787
14 Train Loss 215.86217 Test MSE 236.3503523052192 Test RE 0.2588018030100431 Lambda1 0.16839309
15 Train Loss 210.60922 Test MSE 231.6910974976724 Test RE 0.25623818179594854 Lambda1 0.19370836
16 Train Loss 206.18742 Test MSE 223.5694260727415 Test RE 0.2517070481654477 Lambda1 0.22285025
17 Train Loss 199.9756 Test MSE 217.15349765426032 Test RE 0.24806905089976058 Lambda1 0.25264153
18 Train Loss 193.65712 Test MSE 207.58218888365315 Test RE 0.24254046965144874 Lambda1 0.2822878
19 Train Loss 188.03427 Test MSE 199.22556928599013 Test RE 0.23760835568133606 Lambda1 0.3081943
20 Train Loss 181.83109 

In [14]:
import scipy.io as sio

In [22]:
for tune_reps in range(20):
    label = "inv_HT_stan_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["lambda1"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   -0.8243969
1   -0.15307483
2   0.53936565
3   0.5795185
4   -1.4561002
5   0.12198722
6   0.7041
7   0.68127084
8   -1.4134965
9   0.20109919
10   0.64974755
11   0.8142306
12   -2.1422653
13   0.05638336
14   0.9153641
15   0.63197887
16   -1.7122774
17   [[ 1.45513816e-02 -3.52495834e-02 -2.23435327e-01  6.15948671e-03
  -1.92813063e-03  1.92751712e-03 -8.79180589e-05  4.98717593e-04
  -4.48849169e-04 -4.14079666e-04 -1.23494596e-04  1.04078674e-04
   8.81296073e-05  1.07403735e-04  1.13843482e-04  1.36531773e-04
   2.01167524e-04  1.84232907e-04  2.84948590e-04  2.98798113e-04
   3.42695421e-04  3.72112350e-04  3.99220240e-04  4.01747850e-04
   4.43779369e-04  4.78341739e-04  4.86237754e-04  5.83171262e-04
   6.11778407e-04  6.37944904e-04  7.34501111e-04  7.13067362e-04
   7.13871908e-04  8.22813134e-04  8.24221352e-04  8.91324366e-04
   8.95273814e-04  7.86903140e-04  7.60168361e-04  7.90642574e-04
   8.37648404e-04  8.49442906e-04  8.64006288e-04  8.80231732e-04
   9.87075153

In [23]:
lrb_tune[14]

array([0.5, 0.5])